In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

In [3]:
#pip install openpyxl

## **Carga de datos (Fase 0)**

#### Datos 2023-2024

In [4]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('data/Totalizadores Planta de Cerveza 2023_2024.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs_2023_2024 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2023_2024[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

--- Resumen de hojas y columnas ---
                         hoja  filas  columnas  \
0             Consolidado KPI  12010       125   
1                       Metas     48        57   
2      Consolidado Produccion  12011        19   
3    Totalizadores Produccion  12009        41   
4              Consolidado EE  12011        24   
5       Totalizadores Energia  12009        54   
6            Consolidado Agua  12011        24   
7          Totalizadores Agua  12009        44   
8        Consolidado GasVapor  12011        20   
9   Totalizadores Gas y Vapor  12009        24   
10           Consolidado Aire  12011        14   
11         Totalizadores Aire  12009        12   
12          Totalizadores CO2  12009         9   
13    Totalizadores Efluentes  12009         9   
14       Totalizadores Glicol  12009         8   
15            Seguimiento Dia  12009         4   
16                   Auxiliar  12011        38   
17          Kw Frio  Hl Mosto  12372        15   

             

### **Los dias faltantes son:**
- 31-3-2023
- 31-5-2023
- 31-10-2023
- desde 31-12-2023 hasta 30-6-24


### **Creacion del Diccionario**

Una vez cargada toda la hoja de datos del 2023/2024 en un diccionario, visto los días faltantes y los días en los que no se cargó la última hora (23:59), vamos a crear un diccionario con todos los df con las filas que tengan la última hora de cada día ordenado por orden cronológico. 

In [5]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

def _to_date(x):
    try:
        return pd.to_datetime(x, errors="coerce").date()
    except Exception:
        return pd.NaT

def _to_minutes(x):
    if pd.isna(x):
        return -1
    ts = pd.to_datetime(x, errors="coerce")
    if pd.notna(ts):
        return int(ts.hour) * 60 + int(ts.minute)
    try:
        h = int(float(str(x).replace(",", ".")))
        if 0 <= h <= 23:
            return h * 60
    except Exception:
        pass
    return -1

dfs_23_24 = {}
hojas_saltadas = []

for hoja, df in dfs_2023_2024.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)

    # Filtramos filas sin día y agregamos orden determinístico
    tmp = tmp.dropna(subset=["_dia"]).copy()
    if tmp.empty:
        hojas_saltadas.append((hoja, "Sin días válidos"))
        continue

    tmp["_ord"] = np.arange(len(tmp))  # <- evita usar el índice en sort_values

    # Orden por día, hora (minutos) y orden original
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # Última fila por día (la mayor "_mins"; si empata, la última por "_ord")
    ultimas = tmp.groupby("_dia", as_index=False, sort=True).tail(1)

    # Limpieza de columnas auxiliares y orden final
    ultimas = ultimas.drop(columns=["_dia", "_mins", "_ord"]).sort_values(DAY_COL).reset_index(drop=True)

    dfs_23_24[hoja] = ultimas

### **4 filas con última hora distinta de 23:59**
- 2023-02-28 -> 23:00:00    
- 2023-04-13 -> 19:00:00    
- 2023-04-19 -> 16:00:00    
- 2024-10-26 -> 07:00:00    

### **Interpolación**

Vamos a interpolar los 5 días faltantes:

In [6]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

def completar_e_interpolar_diario(df, day_col=DAY_COL, hour_col=HOUR_COL, hora_por_defecto="23:59:00"):
    g = df.copy()

    # --- fecha como datetime (normalizada al día) ---
    g["_fecha"] = pd.to_datetime(g[day_col], errors="coerce", dayfirst=True).dt.normalize()
    g = g.dropna(subset=["_fecha"]).sort_values("_fecha").drop_duplicates("_fecha", keep="last")

    # --- índice continuo día a día (agrega los días faltantes) ---
    idx_full = pd.date_range(g["_fecha"].min(), g["_fecha"].max(), freq="D")
    g = g.set_index("_fecha").reindex(idx_full)

    # --- reconstruir columnas de fecha/hora ---
    g[day_col] = g.index.date
    if hour_col in g.columns:
        g[hour_col] = g[hour_col].fillna(hora_por_defecto)
    else:
        g[hour_col] = hora_por_defecto

    # --- interpolación SOLO en columnas numéricas ---
    num_cols = g.select_dtypes(include="number").columns
    if len(num_cols):
        # usa el índice temporal para interpolar; luego rellena bordes
        g[num_cols] = g[num_cols].interpolate(method="time").ffill().bfill()

    return g.reset_index(drop=True)

# Aplicarlo a TODO el diccionario (una hoja por vez)
for nombre, df in dfs_23_24.items():
    dfs_23_24[nombre] = completar_e_interpolar_diario(df)

In [7]:
DAY_COL = "DIA"
inicio  = pd.Timestamp("2023-12-31")
fin     = pd.Timestamp("2024-06-30")

for nombre, df in dfs_23_24.items():
    if DAY_COL not in df.columns or df.empty:
        continue

    # Normalizar a fecha y construir máscara para CONSERVAR lo que queda fuera del rango
    fechas = pd.to_datetime(df[DAY_COL], dayfirst=True, errors="coerce").dt.normalize()

    # Rango INCLUSIVO: elimina 31/12/2023 ... 30/06/2024
    mask_keep = (fechas < inicio) | (fechas > fin) | fechas.isna()

    dfs_23_24[nombre] = df.loc[mask_keep].reset_index(drop=True)

#### Datos 2022-2023

Repetimos todo el proceso para los datos del 2022-2023

In [8]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('data/Totalizadores Planta de Cerveza - 2022_2023.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs_2022_2023 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2022_2023[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

--- Resumen de hojas y columnas ---
                         hoja  filas  columnas  \
0             Consolidado KPI  15317       123   
1                       Metas     36        57   
2      Consolidado Produccion  15450        14   
3    Totalizadores Produccion  15316        40   
4              Consolidado EE  15450        24   
5       Totalizadores Energia  15316        59   
6            Consolidado Agua  15451        24   
7          Totalizadores Agua  15316        43   
8        Consolidado GasVapor  15460        20   
9   Totalizadores Gas y Vapor  15316        23   
10           Consolidado Aire  15649        14   
11         Totalizadores Aire  15316        11   
12          Totalizadores CO2  15317         8   
13    Totalizadores Efluentes  15316         8   
14       Totalizadores Glicol  15316         7   
15            Seguimiento Dia  15316         5   
16                   Auxiliar  15754        34   
17          Kw Frio  Hl Mosto   6352        15   

             

### **Los dias faltantes son:**
- 31-3-2023
- 31-5-2023
- 31-10-2023
- desde 07-03-2023 hasta 30-06-23

### **Creacion del diccionario**

In [9]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

dfs_22_23 = {}
hojas_saltadas = []

for hoja, df in dfs_2022_2023.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)

    # Filtramos filas sin día y agregamos orden determinístico
    tmp = tmp.dropna(subset=["_dia"]).copy()
    if tmp.empty:
        hojas_saltadas.append((hoja, "Sin días válidos"))
        continue

    tmp["_ord"] = np.arange(len(tmp))  # <- evita usar el índice en sort_values

    # Orden por día, hora (minutos) y orden original
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # Última fila por día (la mayor "_mins"; si empata, la última por "_ord")
    ultimas = tmp.groupby("_dia", as_index=False, sort=True).tail(1)

    # Limpieza de columnas auxiliares y orden final
    ultimas = ultimas.drop(columns=["_dia", "_mins", "_ord"]).sort_values(DAY_COL).reset_index(drop=True)

    dfs_22_23[hoja] = ultimas

**4 filas con última hora distinta de 23:59**
- 2022-03-02 -> 23:00:00    
- 2022-07-13 -> 23:00:00    
- 2023-02-28 -> 23:00:00    
- 2023-03-06 -> 08:00:00    

### **Interpolación**

In [10]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

# Aplicarlo a TODO el diccionario (una hoja por vez)
for nombre, df in dfs_22_23.items():
    dfs_22_23[nombre] = completar_e_interpolar_diario(df)

In [11]:
DAY_COL = "DIA"
inicio  = pd.Timestamp("2023-03-07")
fin     = pd.Timestamp("2023-06-30")

for nombre, df in dfs_22_23.items():
    if DAY_COL not in df.columns or df.empty:
        continue

    # Normalizar a fecha y construir máscara para CONSERVAR lo que queda fuera del rango
    fechas = pd.to_datetime(df[DAY_COL], dayfirst=True, errors="coerce").dt.normalize()

    # Rango INCLUSIVO: elimina 31/12/2023 ... 30/06/2024
    mask_keep = (fechas < inicio) | (fechas > fin) | fechas.isna()

    dfs_22_23[nombre] = df.loc[mask_keep].reset_index(drop=True)

#### Datos 2021-2022

In [12]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('data/Totalizadores Planta de Cerveza 2021_2022.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs_2021_2022 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2021_2022[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])


--- Resumen de hojas y columnas ---
                         hoja  filas  columnas  \
0             Consolidado KPI  16049        62   
1      Consolidado Produccion  15573        12   
2    Totalizadores Produccion  15573        40   
3              Consolidado EE  16054        21   
4       Totalizadores Energia  15575        53   
5            Consolidado Agua  16049        18   
6          Totalizadores Agua  15575        43   
7        Consolidado GasVapor  16049        17   
8   Totalizadores Gas y Vapor  15575        22   
9            Consolidado Aire  15912        14   
10         Totalizadores Aire  15575        11   
11          Totalizadores CO2  15575         8   
12    Totalizadores Efluentes  15575         8   
13       Totalizadores Glicol  15563         8   
14            Seguimiento Dia  15596         4   
15                   Auxiliar  15754        34   

                                     nombres_columnas  
0   DIA, HORA, EE Planta / Hl, EE Elaboracion / Hl...  
1

### **Dias faltantes:**
- 31-03-2021
- 31-05-2021
- 31-10-2021
- 31-12-2021
- desde 17-03-2022 hasta 30-06-2022
- 31-10-2022

### **Creacion del diccionario:**

In [13]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

dfs_21_22 = {}
hojas_saltadas = []

for hoja, df in dfs_2021_2022.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)

    # Filtramos filas sin día y agregamos orden determinístico
    tmp = tmp.dropna(subset=["_dia"]).copy()
    if tmp.empty:
        hojas_saltadas.append((hoja, "Sin días válidos"))
        continue

    tmp["_ord"] = np.arange(len(tmp))  # <- evita usar el índice en sort_values

    # Orden por día, hora (minutos) y orden original
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # Última fila por día (la mayor "_mins"; si empata, la última por "_ord")
    ultimas = tmp.groupby("_dia", as_index=False, sort=True).tail(1)

    # Limpieza de columnas auxiliares y orden final
    ultimas = ultimas.drop(columns=["_dia", "_mins", "_ord"]).sort_values(DAY_COL).reset_index(drop=True)

    dfs_21_22[hoja] = ultimas

### **3 filas con última hora distinta de 23:59:**
- 2022-03-02 -> 23:00:00
- 2022-03-16 -> 07:00:00	
- 2022-07-13 -> 23:00:00

### **Interpolacioón:**

In [14]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

# Aplicarlo a TODO el diccionario (una hoja por vez)
for nombre, df in dfs_21_22.items():
    dfs_21_22[nombre] = completar_e_interpolar_diario(df)

In [15]:
DAY_COL = "DIA"
inicio  = pd.Timestamp("2022-03-17")
fin     = pd.Timestamp("2022-06-30")

for nombre, df in dfs_21_22.items():
    if DAY_COL not in df.columns or df.empty:
        continue

    # Normalizar a fecha y construir máscara para CONSERVAR lo que queda fuera del rango
    fechas = pd.to_datetime(df[DAY_COL], dayfirst=True, errors="coerce").dt.normalize()

    # Rango INCLUSIVO: elimina 31/12/2023 ... 30/06/2024
    mask_keep = (fechas < inicio) | (fechas > fin) | fechas.isna()

    dfs_21_22[nombre] = df.loc[mask_keep].reset_index(drop=True)

## **Análisis descriptivo y versionado de datos**

Podemos ver que los datos comparten muchos días, por lo que son datos duplicados. Por eso vamos a crear un solo data frame que tenga todos los datos ordenados cronológicamente una sola vez.

#### Checksum (no sabemos donde ponerlo)

In [27]:
# --- Checksum Calculation Block ---
import hashlib
import json
import os

print("\\n--- Calculando Checksum de Datos Crudos ---")

# --- Paso 2: Reunir todos los DataFrames crudos ---
lista_dfs_crudos = []
if 'dfs_2023_2024' in locals():
    lista_dfs_crudos.extend(dfs_2023_2024.values())
if 'dfs_2022_2023' in locals():
    lista_dfs_crudos.extend(dfs_2022_2023.values())
if 'dfs_2021_2022' in locals():
    lista_dfs_crudos.extend(dfs_2021_2022.values())

print(f"Total de hojas (DataFrames) crudos encontrados: {len(lista_dfs_crudos)}")

if not lista_dfs_crudos:
    print("¡Advertencia! No se encontraron DataFrames crudos para calcular el checksum.")
else:
    # --- Paso 3: Unificar todo ---
    # Usamos sort=False para eficiencia, ordenaremos después explícitamente
    df_crudo_total = pd.concat(lista_dfs_crudos, ignore_index=True, sort=False)
    print(f"DataFrame crudo unificado tiene {df_crudo_total.shape[0]} filas y {df_crudo_total.shape[1]} columnas.")

    # --- Paso 4: ¡Ordenar! ---
    # Asumimos que 'HORA' es la columna de timestamp más fiable en los datos crudos
    columna_orden = 'HORA'
    if columna_orden in df_crudo_total.columns:
        print(f"Ordenando datos crudos por '{columna_orden}'...")
        # Aseguramos que HORA sea datetime para un orden correcto, ignorando errores
        df_crudo_total[columna_orden] = pd.to_datetime(df_crudo_total[columna_orden], errors='coerce')
        # Ordenamos, poniendo NaT (fechas no válidas) al final para consistencia
        df_crudo_ordenado = df_crudo_total.sort_values(by=columna_orden, kind='stable', na_position='last').reset_index(drop=True)
    else:
        print(f"¡Advertencia! No se encontró la columna '{columna_orden}' para ordenar. El checksum podría no ser reproducible.")
        # Como fallback MUY BÁSICO, intentamos ordenar por todas las columnas
        # Esto es lento y menos robusto, pero mejor que nada.
        try:
            df_crudo_ordenado = df_crudo_total.sort_values(by=df_crudo_total.columns.tolist()).reset_index(drop=True)
            print("Fallback: Ordenando por todas las columnas.")
        except Exception as e:
            print(f"Error al intentar ordenar por todas las columnas: {e}. Checksum no se calculará.")
            df_crudo_ordenado = None # Marcamos para no continuar

    if df_crudo_ordenado is not None:
        # --- Paso 5: Convertir a bytes ---
        print("Convirtiendo DataFrame ordenado a bytes...")
        try:
            datos_en_bytes = df_crudo_ordenado.to_csv(index=False, encoding='utf-8').encode('utf-8')
        except Exception as e:
            print(f"Error al convertir a CSV/bytes: {e}. Usando representación de string como fallback.")
            # Fallback muy simple si to_csv falla (raro)
            datos_en_bytes = str(df_crudo_ordenado.to_dict(orient='records')).encode('utf-8')


        # --- Paso 6: Calcular Hash MD5 ---
        print("Calculando hash MD5...")
        hash_md5 = hashlib.md5(datos_en_bytes).hexdigest()
        print(f"¡Checksum MD5 de datos crudos!: {hash_md5}")

        # --- Paso 7: Guardar ---
        ruta_checksum = os.path.join('data', 'checksums.json')
        checksum_info = {
            'datos_crudos_unificados_v2': hash_md5 # v2 para diferenciar si ya había uno
        }
        try:
            with open(ruta_checksum, 'w') as f:
                json.dump(checksum_info, f, indent=4)
            print(f"Checksum guardado exitosamente en: {ruta_checksum}")
        except Exception as e:
            print(f"Error al guardar el checksum en {ruta_checksum}: {e}")

# --- Fin del Checksum Block ---

\n--- Calculando Checksum de Datos Crudos ---
Total de hojas (DataFrames) crudos encontrados: 52
DataFrame crudo unificado tiene 708963 filas y 438 columnas.
Ordenando datos crudos por 'HORA'...


C:\Users\angim\AppData\Local\Temp\ipykernel_19092\1030262672.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_crudo_total[columna_orden] = pd.to_datetime(df_crudo_total[columna_orden], errors='coerce')


Convirtiendo DataFrame ordenado a bytes...
Calculando hash MD5...
¡Checksum MD5 de datos crudos!: f7b8d7c5153f2dd7e0ddac7d4fe1a435
Checksum guardado exitosamente en: data\checksums.json


Este df_crudo_total es temporal. Solo existe dentro de ese bloque de código con el único propósito de ser "sellado". No lo usamos para el resto de tu análisis (tú sigues usando tus diccionarios separados, ¡lo cual está perfecto!).

El checksums.json que guardamos en data/ es ese "sello". Cualquiera (¡incluyéndome a mí!) puede descargar tus 3 Excels, correr ese mismo bloque de código, y si el hash que obtenemos es idéntico al que tú guardaste, tenemos 100% de certeza de que estamos trabajando con exactamente los mismos datos crudos.


**Ficha-resumen**
- ¿Qué es df_crudo_total ? Es un DataFrame temporal que une TODOS los datos crudos de TODOS tus archivos y hojas.
- ¿Por qué lo creamos? Para poder generar UN ÚNICO "sello de garantía" (checksum) que represente el 100% de tus datos de origen. Es el "Tesoro Completo".
- ¿Por qué es importante? Permite que cualquier persona (tu colega, tu profesor) verifique con un solo comando si sus archivos Excel son exactamente idénticos a los tuyos. Esto se llama Integridad de Datos y es un pilar de la Reproducibilidad (MLOps).
- ¿Cuál es el paso más importante? df.sort_values(by='HORA') . Sin ordenar los datos, dos personas con los mismos archivos podrían obtener hashes diferentes, y el checksum no serviría para nada.

In [16]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

RANGOS = {
    "dfs_21_22": [("2021-01-01", "2021-12-31")],
    "dfs_22_23": [("2022-01-01", "2022-12-31")],
    "dfs_23_24": [("2023-01-01", "2023-12-30"),
                  ("2024-07-01", "2024-10-26")],
}

HOJAS_INCLUIR = [
    "Consolidado KPI", "Consolidado Produccion", "Totalizadores Produccion", "Consolidado EE", "Totalizadores Energia",
    "Consolidado Agua", "Totalizadores Agua", "Consolidado GasVapor", "Totalizadores Gas y Vapor", "Consolidado Aire",
    "Totalizadores Aire", "Totalizadores Efluentes", "Totalizadores Glicol", "Totalizadores CO2"
]

DICS = {
    "dfs_21_22": dfs_21_22,
    "dfs_22_23": dfs_22_23,
    "dfs_23_24": dfs_23_24,
}

def slice_por_fecha(df, start, end, day_col=DAY_COL):
    if df.empty or day_col not in df.columns:
        return df.iloc[0:0]
    fechas = pd.to_datetime(df[day_col], errors="coerce", dayfirst=True).dt.normalize()
    mask = fechas.between(pd.to_datetime(start), pd.to_datetime(end), inclusive="both")
    return df.loc[mask].copy()

def ordenar_crono(df, day_col=DAY_COL, hour_col=HOUR_COL):
    if df.empty:
        return df
    dia = pd.to_datetime(df[day_col], errors="coerce", dayfirst=True)
    if hour_col in df.columns:
        dt = pd.to_datetime(dia.dt.date.astype(str) + " " + df[hour_col].astype(str),
                            errors="coerce", dayfirst=True)
    else:
        dt = dia
    return (df.assign(_dt=dt)
              .sort_values("_dt", kind="stable", na_position="last")
              .drop(columns="_dt").reset_index(drop=True))


partes_por_hoja = defaultdict(list)

for nombre_dic, dic in DICS.items():
    rangos = RANGOS.get(nombre_dic, [])
    for (inicio, fin) in rangos:
        for hoja, df in dic.items():
            if HOJAS_INCLUIR and hoja not in HOJAS_INCLUIR:
                continue
            recorte = slice_por_fecha(df, inicio, fin)
            if not recorte.empty:
                partes_por_hoja[hoja].append(recorte)

dfs_completo = {}
for hoja, partes in partes_por_hoja.items():
    # Unificar columnas: las faltantes quedan como NaN
    todas_cols = list(set().union(*(p.columns for p in partes)))
    partes_alineadas = [p.reindex(columns=todas_cols) for p in partes]
    combinado = pd.concat(partes_alineadas, ignore_index=True, sort=False)

    # Orden temporal final
    if DAY_COL in combinado.columns:
        combinado = ordenar_crono(combinado, DAY_COL, HOUR_COL)
    dfs_completo[hoja] = combinado

In [17]:
dfs_completo['Consolidado Produccion'] = dfs_completo['Consolidado Produccion'].drop(columns="Fecha/Hora", errors="ignore")
dfs_completo['Consolidado EE'] = dfs_completo['Consolidado EE'].drop(columns=['Fecha/Hora', 'Kw de Frio'], errors="ignore")

In [18]:
PATRONES_DIA = ("dia", "Dia", "DIA")

def detectar_col_dia(df, patrones=PATRONES_DIA):
    cols = [str(c) for c in df.columns]
    cand = [c for c in cols if any(p in c.lower() for p in patrones)]
    if not cand:
        raise ValueError("No se encontró columna de día/fecha en un DF.")
    # Heurística: prioriza nombres más específicos
    preferencia = ["dia", "día", "Dia", "DIA"]
    cand_orden = sorted(cand, key=lambda c: next((i for i,p in enumerate(preferencia) if p in c.lower()), 99))
    return cand_orden[0]

def normalizar_dia_col(df, col_dia):
    out = df.copy(deep=True)
    out[col_dia] = pd.to_datetime(out[col_dia], errors="coerce")
    # Si trae hora, nos quedamos con la fecha (día civil)
    out[col_dia] = out[col_dia].dt.normalize()
    # Renombramos a un nombre canónico común
    if col_dia != "dia":
        out = out.rename(columns={col_dia: "dia"})
    return out

def deduplicar_por_dia(df):
    # Si hay múltiples filas por día en un DF, evitamos explosiones en los merges
    # Estrategia simple: nos quedamos con la ultima por día (ajusta si necesitas otra agregación)
    if df.duplicated("dia").any():
        df = df.sort_values("dia").drop_duplicates("dia", keep="last")
    return df

def mergear_por_dia(dfs_completo):
    dfs_norm = []
    for k, df in dfs_completo.items():
        col = detectar_col_dia(df)
        tmp = normalizar_dia_col(df, col)
        tmp = deduplicar_por_dia(tmp)
        # Evita choques de nombres: agrega sufijo con la clave del dict a las columnas no-clave
        cols_no_clave = [c for c in tmp.columns if c != "dia"]
        tmp = tmp[["dia"] + cols_no_clave].add_suffix(f"__{k}")
        tmp = tmp.rename(columns={f"dia__{k}": "dia"})
        dfs_norm.append(tmp)

    # Merge iterativo (outer) por 'dia'
    from functools import reduce
    df_unificado = reduce(lambda l, r: pd.merge(l, r, on="dia", how="outer"), dfs_norm)

    # Orden final
    df_unificado = df_unificado.sort_values("dia").reset_index(drop=True)
    return df_unificado

df_unificado = mergear_por_dia(dfs_completo)

Verificación de dimensiones del dataset, tipos de variables y rangos de valores

In [19]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Columns: 425 entries, dia to Tot L3, L4 y Planta de CO2__Totalizadores Glicol
dtypes: datetime64[ns](1), float64(401), object(23)
memory usage: 3.9+ MB


In [20]:
df_unificado.describe()

,dia,Meta ET Servicios__Consolidado KPI,Agua Envas / Hl__Consolidado KPI,Meta Aire Bodega__Consolidado KPI,Agua Paste L3 / Hl__Consolidado KPI,Meta EE Resto Serv__Consolidado KPI,Meta EE Linea 2__Consolidado KPI,Meta Aire Cocina__Consolidado KPI,Meta Agua Planta de Agua__Consolidado KPI,EE Resto Serv / Hl__Consolidado KPI,...,Id__Totalizadores Glicol,Tot A130/330/430__Totalizadores Glicol,Tot Trasiego__Totalizadores Glicol,Tot L3. L4 y Planta de CO2__Totalizadores Glicol,Tot Reposo Inferior__Totalizadores Glicol,Tot Fermantacion_Cocina__Totalizadores Glicol,Tot A10/20__Totalizadores Glicol,Tot A40/240/50/60/Centec/Filtro__Totalizadores Glicol,Tot Reposo Superior__Totalizadores Glicol,"Tot L3, L4 y Planta de CO2__Totalizadores Glicol"
count,1213,847.000000,1212.000000,847.000000,847.000000,847.000000,847.000000,847.000000,847.000000,1212.000000,...,847.000000,847.000000,1212.000000,365.000000,365.000000,365.000000,847.000000,847.000000,365.000000,482.000000
mean,2022-08-31 10:42:14.542456832,3.812812,1.209606,1.517460,0.986882,0.455238,2.630265,0.505820,14.788598,0.333766,...,26792.167651,4844.695404,1460.432296,1209.291173,5675.086397,1383.276041,2114.529681,2919.339250,135.887096,3604.845039
min,1970-01-01 00:00:00,3.368182,0.003756,1.307143,-0.018261,0.392143,2.265714,0.435714,12.857143,-707.419891,...,10024.000000,0.000000,0.000000,0.000000,170.630000,224.910000,0.000000,0.000000,0.000000,1.312500
25%,2021-10-30 00:00:00,3.486364,0.733096,1.371429,0.000000,0.411429,2.377143,0.457143,14.142857,0.445170,...,15333.500000,4161.860000,587.487500,606.090000,4410.960000,600.940000,0.000000,1251.420000,1.400000,468.582500
50%,2022-08-29 00:00:00,3.604545,0.980745,1.457143,0.000000,0.437143,2.525714,0.485714,14.571429,0.608472,...,31474.000000,5067.460000,1158.680000,1017.660000,5633.530000,951.280000,0.000000,2086.520000,3.450000,1084.690000
75%,2023-06-28 00:00:00,4.018182,1.333608,1.660714,0.097289,0.498214,2.878571,0.553571,15.357143,0.887435,...,36776.500000,5972.937500,1688.047500,1443.190000,6959.130000,1515.620000,4870.500000,3933.295000,12.600000,5078.165000
max,2024-10-26 00:00:00,4.963636,55.821429,1.862143,131.700000,0.558643,3.227714,0.620714,17.871429,202.678571,...,42008.000000,11548.220000,296218.410000,9275.440000,11066.500000,10870.690000,11548.220000,12927.960000,4054.270000,27378.380000
std,NaN,0.489438,1.913240,0.162919,8.839278,0.048876,0.282393,0.054306,0.976547,21.434228,...,11083.098224,2110.611838,8515.193813,1008.849217,2003.208708,1452.803698,2629.050309,2230.783666,561.474599,4977.023500


In [21]:
df_unificado.select_dtypes(include=["object"]).columns.tolist()

['HORA__Consolidado KPI',
 'HORA__Consolidado Produccion',
 'DIA__Consolidado Produccion',
 'HORA__Totalizadores Produccion',
 'Nivel Silo Bagazo Norte (1)__Totalizadores Produccion',
 'DIA__Totalizadores Produccion',
 'HORA__Consolidado EE',
 'HORA__Totalizadores Energia',
 'KW Trafo 8__Totalizadores Energia',
 'HORA__Consolidado Agua',
 'HORA__Totalizadores Agua',
 'HORA__Consolidado GasVapor',
 'HORA__Totalizadores Gas y Vapor',
 'HORA__Consolidado Aire',
 'HORA__Totalizadores Aire',
 'HORA__Totalizadores CO2',
 'Totalizador Bba P4__Totalizadores Efluentes',
 'HORA__Totalizadores Efluentes',
 'Totalizador Bba P51__Totalizadores Efluentes',
 'Totalizador Bba P2__Totalizadores Efluentes',
 'Totalizador Bba Envasado__Totalizadores Efluentes',
 'Totalizador Bba P1__Totalizadores Efluentes',
 'HORA__Totalizadores Glicol']

In [22]:
cols_float = [
    "Nivel Silo Bagazo Norte (1)__Totalizadores Produccion",
    "Totalizador Bba P51__Totalizadores Efluentes",
    "Totalizador Bba P2__Totalizadores Efluentes",
    "Totalizador Bba P4__Totalizadores Efluentes",
    "Totalizador Bba Envasado__Totalizadores Efluentes",
    "Totalizador Bba P1__Totalizadores Efluentes",
    "KW Trafo 8__Totalizadores Energia",
]

df_unificado[cols_float] = (df_unificado[cols_float].astype(str).apply(lambda s: s.str.replace(r"\.", "", regex=True).str.replace(",", ".", regex=False))
      .apply(pd.to_numeric, errors="coerce").astype("float64")
)

In [23]:
cols_hora = [
    'HORA__Consolidado KPI',
    'HORA__Consolidado Produccion',
    'HORA__Totalizadores Produccion',
    'HORA__Consolidado EE',
    'HORA__Totalizadores Energia',
    'HORA__Consolidado Agua',
    'HORA__Totalizadores Agua',
    'HORA__Consolidado GasVapor',
    'HORA__Totalizadores Gas y Vapor',
    'HORA__Consolidado Aire',
    'HORA__Totalizadores Aire',
    'HORA__Totalizadores CO2',
    'HORA__Totalizadores Efluentes',
    'HORA__Totalizadores Glicol',
]

# 1) Igualdad exacta columna a columna (por fila), tratando NaN como iguales
base = df_unificado[cols_hora[0]].fillna("__NA__")
iguales_mask = df_unificado[cols_hora].fillna("__NA__").eq(base, axis=0)

# 2) ¿Todas las columnas son iguales en todas las filas?
todas_iguales = bool(iguales_mask.all().all())
print("¿Todas las HORA__ son iguales en todas las filas?:", todas_iguales)

# 3) Filas donde NO coinciden todas
filas_ok = iguales_mask.all(axis=1)
diff_rows = df_unificado.loc[~filas_ok, cols_hora]
print("Filas con diferencias:", len(diff_rows))

print(diff_rows.head(10))

¿Todas las HORA__ son iguales en todas las filas?: False
Filas con diferencias: 1213
  HORA__Consolidado KPI HORA__Consolidado Produccion  \
0                   NaN                     07:00:00   
1              23:59:00                          NaN   
2              23:59:00                          NaN   
3              23:59:00                          NaN   
4              23:59:00                          NaN   
5              23:59:00                          NaN   
6              23:59:00                          NaN   
7              23:59:00                          NaN   
8              23:59:00                          NaN   
9              23:59:00                          NaN   

  HORA__Totalizadores Produccion HORA__Consolidado EE  \
0                       07:00:00                  NaN   
1                            NaN             23:59:00   
2                            NaN             23:59:00   
3                            NaN             23:59:00   
4            

In [24]:
# crea la columna HORA
df_unificado["HORA"] = pd.to_datetime(df_unificado["HORA__Consolidado KPI"], errors="coerce")

# elimina todas las columnas de hora originales
df_unificado = df_unificado.drop(columns=[c for c in cols_hora if c in df_unificado.columns])

C:\Users\angim\AppData\Local\Temp\ipykernel_19092\3562094624.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_unificado["HORA"] = pd.to_datetime(df_unificado["HORA__Consolidado KPI"], errors="coerce")


Identificación y cuantificación de valores faltantes, detección de valores atípicos y errores

In [25]:
df_unificado.isnull().sum().sort_values(ascending=False).head(25)

Unnamed: 66__Consolidado KPI                       1213
Unnamed: 72__Consolidado KPI                       1213
Unnamed: 46__Consolidado KPI                       1213
Unnamed: 124__Consolidado KPI                      1213
Unnamed: 67__Consolidado KPI                       1213
EE Linea 5 / Hl__Consolidado KPI                   1213
Unnamed: 123__Consolidado KPI                      1213
Unnamed: 22__Consolidado KPI                       1213
Unnamed: 21__Consolidado KPI                       1213
Hl Reserva 7__Totalizadores Produccion             1213
Unnamed: 65__Consolidado KPI                       1213
Unnamed: 113__Consolidado KPI                      1213
Unnamed: 35__Consolidado KPI                       1213
 __Consolidado KPI                                 1213
Unnamed: 14__Consolidado Produccion                1213
Unnamed: 103__Consolidado KPI                      1213
Unnamed: 84__Consolidado KPI                       1213
HL Mosto Indio__Totalizadores Produccion        

In [26]:
# Cuenta ceros
zero_counts = (df_unificado.eq(0) | df_unificado.eq("0")).sum(axis=0)
zero_ratio  = ((zero_counts / len(df_unificado)) * 100).sort_values(ascending=False)

umbral = 75.0
muchos_ceros = zero_ratio[zero_ratio >= umbral]

print("\nColumnas con ≥", umbral, "% ceros:", len(muchos_ceros), 'de', len(df_unificado.columns), "columnas totales")
print()
print(muchos_ceros.apply(lambda x: f"{x:.2f}%"))


Columnas con ≥ 75.0 % ceros: 18 de 412 columnas totales

VAPOR DE CALDERA 1 KG__Totalizadores Gas y Vapor           99.92%
Tot_Vapor_Caldera 3__Totalizadores Gas y Vapor             99.92%
Fuel Oil Tk1 (Kg)__Totalizadores Gas y Vapor               99.92%
Fuel Oil Tk2 (Kg)__Totalizadores Gas y Vapor               99.92%
Agua Filt FMaCist CE__Totalizadores Agua                   99.92%
Red Barriles__Totalizadores Agua                           99.92%
Kw llum/Serv L2__Totalizadores Energia                     99.92%
Rep Agua Cist CE__Totalizadores Agua                       99.92%
KW Linea 4__Totalizadores Energia                          99.92%
Totalizador Bba Gas__Totalizadores Efluentes               99.84%
Glicol Paste L3__Totalizadores Agua                        99.01%
TOT GAS ENTRADA PRINCIPAL M3__Totalizadores Gas y Vapor    98.27%
KW Atlas 3__Totalizadores Energia                          91.59%
FC Lavadora L2__Consolidado Agua                           85.33%
FC Lavadora L2__To

In [29]:
# Parámetros
# Usar los nombres de columnas tal como aparecen en los DataFrames crudos
DAY_COL  = "DIA"
HOUR_COL = "HORA"
VAL_COL  = "Frio (Kw)"
EPS = 1e-6  # tolerancia numérica

def _to_minutes(h):
    # admite "HH:MM" o "HH:MM:SS"
    try:
        parts = str(h).split(":")
        hh, mm = int(parts[0]), int(parts[1])
        ss = int(parts[2]) if len(parts) > 2 else 0
        return hh*60 + mm + ss/60
    except Exception:
        return np.nan

# 1) Tomamos el df
df = dfs_2023_2024["Consolidado EE"].copy()

# 2) Normalizamos fecha/hora y ordenamos
df["_dia"]  = pd.to_datetime(df[DAY_COL], errors="coerce", dayfirst=True).dt.date
df["_mins"] = df[HORA].map(_to_minutes) if HOUR_COL not in df.columns and "HORA" in df.columns else df[HOUR_COL].map(_to_minutes)
df = df.dropna(subset=["_dia","_mins"]).sort_values(["_dia","_mins"])

# 3) Chequeos por día
res = []
for dia, g in df.groupby("_dia", sort=True):
    s = pd.to_numeric(g.get(VAL_COL, pd.Series([], dtype="float")), errors="coerce").fillna(0.0).values
    if len(s) == 0:
        continue
    bad_reset     = not (abs(s[0]) <= EPS)              # el primer valor del día no es ~0
    bad_monotone  = (pd.Series(s).diff().fillna(0) < -EPS).any()  # hay caídas dentro del día
    if bad_reset or bad_monotone:
        res.append({"dia": dia, "mal_reset": bad_reset, "mal_monotonia": bad_monotone})

mal_df = pd.DataFrame(res).sort_values("dia") if res else pd.DataFrame(columns=["dia", "mal_reset", "mal_monotonia"])

print("DÍAS CON PROBLEMAS EN 'Frio (Kw)':")
print(mal_df if not mal_df.empty else "✓ Todo OK (acumulado correcto y reseteo diario).")

DÍAS CON PROBLEMAS EN 'Frio (Kw)':
          dia  mal_reset  mal_monotonia
0  2023-04-30      False           True
1  2023-05-20      False           True
2  2023-06-30      False           True
3  2023-07-03      False           True
4  2023-09-30      False           True
5  2023-11-30      False           True
6  2024-07-01       True          False
7  2024-09-30      False           True
